In [1]:
from qiskit import *
from qiskit.circuit.library import UnitaryGate
from qiskit.circuit.library import HamiltonianGate
from qiskit.quantum_info import Statevector
from qiskit_aer import AerSimulator
simulator = AerSimulator()

import numpy as np
from numpy import linalg
import math
import random

from sympy import Matrix
from sympy.physics.quantum import TensorProduct

import matplotlib.pyplot as plt

%run QuantumFourierTransform.ipynb

In [2]:
# Rescale matrix A such that its rows can be encoded as quantum states

def Rescale(A,b):
    A_rescale = []
    b_rescale = []
    for row in range(len(A)):
        length = 0.0
        for entry in range(len(A[row])):
            length = length+(float(A[row][entry])**2)
        length = math.sqrt(length)
        
        temp_A = list(A[row]/length)
        temp_b = list(b[row]/length)
        
        A_rescale.append(temp_A)
        b_rescale.append(temp_b)
    
    new_A = np.array(A_rescale)
    new_b = np.array(b_rescale)
    
    return(new_A, new_b)

In [3]:
# Initializes the algorithm

def Initialize(A,b):
    U = float(max(A.max(),b.max()))
    n = float(len(A[0]))
    
    small_v = n**(-n) * (n*U)**((-n**(2))*(n+1))
    big_V = (2*n)**n *(n*U)**(n**2)
    max_time = math.ceil(2*(n+1)*math.log(big_V/small_v))
    
    radius = big_V/math.pi
    D_0 = radius*np.identity(int(n))
    
    return(max_time, D_0)

In [309]:
# Given vectors a_row, and x_vector, test to see if their inner product in equal to b
def SwapTest(a_row,x_vector,b_target,error):
    # Rescale x_vector and b_target
    x_norm = np.linalg.norm(x_vector)
    x_normalized = x_vector/x_norm
    b_normalized = b_target/x_norm
    
    # Prepair states a_row and x_normalized
    length = int(math.log(len(a_row), 2))
    circuit = QuantumCircuit(2*length+1,1)
    
    circuit.initialize(a_row, range(1,length+1))
    circuit.initialize(x_normalized, range(length+1, 2*length+1))
    
    # Generate Circuit
    circuit.h(0)
    
    for index in range(1,length+1):
        circuit.cswap(0, index, index+length)
    
    circuit.h(0)
    circuit.measure(0,0)
    
    # Perform measurements
    job = transpile(circuit,backend=simulator)
    shots = int(math.ceil(1/(error**2)))
    result = simulator.run(job, shots=shots).result()
    
    # Data Processing
    inner_product = math.sqrt(2*(result.data()['counts']['0x0']/shots-0.5))
    difference = abs(inner_product - b_normalized)
    
    if difference >= error:
        return a_row
    else:
        return "Groovy"

In [316]:
# Given A_matrix, x_vector, and b_vector, will tell which row of A_matrix has been violated, if any
def ViolatedRow(A_matrix,x_vector,b_vector,error):
    for rows in range(len(A)):
        if type(SwapTest(A[rows],x,b[rows],e)) == str:
            pass
        else:
            return A[rows]
        
    return 'Groovy'

In [319]:
# Test Data
A = np.array([[0.8, 0.6, 0.0, 0.0],
              [1.0, 0.0, 0.0, 0.0],
              [0.5, 0.5, 0.5, 0.5]])

x = np.array([3.0, 2.0, 1.0, 0.0])

b = np.array([3.605, 3.0, 3.0])

e = .001

In [320]:
ViolatedRow(A,x,b,e)

array([0.8, 0.6, 0. , 0. ])